# **Projeto Final: Predição da Energia de Ligação dos Excitons utilizando Machine-Learning para materiais 2D.**
## **Objetivo do modelo**
O objetivo do projeto é encontrar um modelo adequado dentre os selecionados, isto é, o que possuir mais acurácia nas suas predições.
Especificamente, queremos um modelo de regressão para prever a **energia de ligação dos excitons** (E_b).
Baseado nos dois últimos projetos feitos, testarei os diferentes modelos utilizando o Gradient Boosting, pois foi o que se saiu melhor para um dos modelos testados.
Os modelos utilizados serão os apresentados em aula mais alguns outros para verificar se há um melhor.

In [1]:
#Importa as bibliotecas
from pymatgen.core.composition import *
import numpy as np
import pandas as pd
import ase.db
import json
import re

In [2]:
#Lê o arquivo para formar o dataframe das propriedades atômicas
df_atoms = pd.read_csv('Schleder2019_AtomicTable.csv')
df_atoms

,Element,Z,Electronegativity,IonizationPotential,ElectronAffinity,HOMO,LUMO,r_s_orbital,r_p_orbital,r_d_orbital,r_atomic_nonbonded,r_valence_lastorbital,r_covalent,Valence,PeriodicColumn,PeriodicColumn_upto18,NumberUnfilledOrbitals,Polarizability
0,H,1,2.20,-12.6833,-1.5273,-6.4925,0.7250,0.3865,NaN,NaN,0.37,0.3865,0.31,1.0,1.0,1.0,1.0,4.507107
1,He,2,NaN,-26.7499,3.0204,-15.7610,1.5714,0.2964,1.0292,0.4176,0.32,0.2964,0.28,2.0,8.0,18.0,0.0,1.383746
2,Li,3,0.98,-5.3606,-0.5863,-2.8744,-0.9074,1.6578,1.8874,2.0869,1.34,1.6578,1.28,1.0,1.0,1.0,1.0,164.000000
3,Be,4,1.57,-9.5007,0.7972,-5.6097,-2.0104,1.0805,1.2128,1.9594,0.90,1.0805,0.96,2.0,2.0,2.0,0.0,37.710000
4,B,5,2.04,-8.1261,0.0312,-3.6067,2.4547,0.8025,0.8348,1.3619,0.82,0.8348,0.84,3.0,3.0,13.0,5.0,20.530000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Fl,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.590000
114,Mc,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,Lv,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,Ts,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Propriedades
prop = ['Z',
        'Electronegativity',
        'IonizationPotential',
        'ElectronAffinity',
        'HOMO',
        'LUMO',
        'r_s_orbital',
        'r_p_orbital',
        'r_d_orbital',
        'r_atomic_nonbonded',
        'r_valence_lastorbital',
        'r_covalent',
        'Valence',
        'PeriodicColumn',
        'PeriodicColumn_upto18',
        'NumberUnfilledOrbitals',
        'Polarizability']

In [4]:
df_atoms.set_index('Element', inplace = True)
dicio = df_atoms.to_dict('index')

In [5]:
data = ase.db.connect('./c2db-2021-06-24.db')
rows = data.select(is_magnetic=False) #Seleciona materias não metálicos e não magnéticos

#Listas de propriedades
lista = []
pesos = []
stch = []

#Features estatísticas
media_interm = {}

#Lista que guarda todas as informações
lista_completa = []

for row in rows:
    try:
        comp = Composition(row.formula).as_dict()
        elem = list(comp.items())
        media_interm['Material'] = row.formula
        media_interm['Space group'] = row.spacegroup
        media_interm['Exciton Binding Energy'] = row.E_B
        media_interm['Stoichiometry'] = row.stoichiometry
        media_interm['Crystal Type'] = row.crystal_type
    
        for i in prop:
            #Lista de propriedades de cada átomo
            for m in range(0, len(elem)):
                lista.append(dicio[elem[m][0]][i])
                pesos.append(elem[m][1])
                if (len(elem)==2):
                    stch.append(row.stoichiometry)
        
            #Valor médio
            media_interm[f'media_{i}'] = np.mean(lista)

    
            #édia ponderada
            avg = np.average(lista,weights=pesos)
            media_interm[f'media_pon_{i}'] = avg
    
            #Valor máximo e mínimo
            max_prop = max(lista)
            min_prop = min(lista)
            media_interm[f'max_{i}'] = max_prop
            media_interm[f'min_{i}'] = min_prop
    
            #Desvio padrão em relação a média
            media_interm[f'desvio_{i}'] = np.std(lista)
    
            #Desvio padrão em relação a média ponderada
            sum_prop = 0
            for j in lista:
                sub2 = (j - avg)**2
                sum_prop = sum_prop + sub2
            media_interm[f'desvio_pon_{i}'] = np.sqrt(sum_prop/len(lista)) 
        
            lista.clear()
            pesos.clear()
        
        lista_completa.append(media_interm.copy())
    except:
        pass

print(set(stch))
    
print(len(lista_completa))
df = pd.DataFrame(lista_completa)
df

{'AB', 'AB2'}
333


,Material,Space group,Exciton Binding Energy,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,As4,Pmna,0.565799,A,A-53-h,33.000000,33.000000,33,33,0.000000,...,3.0,3.0,0.000000,0.000000,29.800000,29.800000,29.80,29.80,0.000000,0.000000
1,Ag2I2,P4/nmm,1.082458,AB,AB-129-bc,50.000000,50.000000,53,47,3.000000,...,1.0,1.0,0.000000,0.000000,43.550000,43.550000,52.50,34.60,8.950000,8.950000
2,Ag2Se2,Pm,1.091100,AB,AB-6-ab,40.500000,40.500000,47,34,6.500000,...,2.0,1.0,0.500000,0.500000,39.370000,39.370000,52.50,26.24,13.130000,13.130000
3,Al2Te2,P-6m2,0.799185,AB,AB-187-hi,32.500000,32.500000,52,13,19.500000,...,5.0,2.0,1.500000,1.500000,46.200000,46.200000,55.40,37.00,9.200000,9.200000
4,GaAs,P3m1,0.397637,AB,AB-156-bc,32.000000,32.000000,33,31,1.000000,...,5.0,3.0,1.000000,1.000000,40.600000,40.600000,51.40,29.80,10.800000,10.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,ISbSe,P3m1,0.472964,ABC,ABC-156-ac,46.000000,46.000000,53,34,8.524475,...,3.0,1.0,0.816497,0.816497,34.463333,34.463333,42.55,26.24,6.659231,6.659231
329,MoSTe,P3m1,0.508695,ABC,ABC-156-ac,36.666667,36.666667,52,16,15.173076,...,6.0,2.0,1.885618,1.885618,47.456667,47.456667,86.00,19.37,28.188599,28.188599
330,WSSe,P3m1,0.506804,ABC,ABC-156-ac,41.333333,41.333333,74,16,24.239545,...,6.0,2.0,1.885618,1.885618,40.203333,40.203333,75.00,19.37,24.764292,24.764292
331,ZrSTe,P3m1,0.471690,ABC,ABC-156-ac,36.000000,36.000000,52,16,14.966630,...,8.0,2.0,2.828427,2.828427,59.123333,59.123333,121.00,19.37,44.341445,44.341445


In [6]:
df.to_csv('dataset_regression_final_full.csv') #Coloca o dataframe em um arquivo csv
data_materials = pd.read_csv('dataset_regression_final_full.csv')

# **Testando diferentes modelos**
Aqui, vamos ver diferentes modelos para determinar qual é o que possui a melhor performance.

## **Linear Regression**

In [7]:
#Importando as bibliotecas para o modelo
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r_2
from sklearn.metrics import mean_absolute_error as mae
import seaborn as sns

In [8]:
#Preparando o dataset para começar o modelo
data_materials.drop('Unnamed: 0', axis = 'columns', inplace = True)
data_materials.dropna(inplace = True)
data_materials.info()

<class 'pandas.core.frame.DataFrame'>
Index: 329 entries, 0 to 332
Columns: 107 entries, Material to desvio_pon_Polarizability
dtypes: float64(101), int64(2), object(4)
memory usage: 277.6+ KB


In [9]:
#Train-Test Split
y = data_materials['Exciton Binding Energy']
x = data_materials.drop(['Material', 'Space group','Exciton Binding Energy', 'Stoichiometry', 'Crystal Type'] , axis = 'columns')

#Criando os dados de teste e treino:
x_train, x_test, y_train, y_test = tts(x, y , test_size = 0.25, random_state = 42)

#Usando o modelo
lsm = LinearRegression()
lsm.fit(x_train,y_train)

#Predição de treino
y_pred_train = lsm.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = lsm.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))
print("Intercept:", lsm.intercept_)

Erro do train set: 0.039289440686179844
Erro do test set: 0.0735567996477835
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.7663237773897361
MSE do modelo: 0.0735567996477835
MAE do modelo: 0.21653852359617773
Intercept: 30.09679070713449


## **Suport Vector Regression**

In [10]:
#Bibliotecas necessárias
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf')
svr.fit(x_train, y_train)

#Predição de treino
y_pred_train = svr.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = svr.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))
print("Intercept:", svr.intercept_)

Erro do train set: 0.12950620205490426
Erro do test set: 0.1435913272872497
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.5438371554941663
MSE do modelo: 0.1435913272872497
MAE do modelo: 0.2924721278957947
Intercept: [1.5803805]


## **Decision Tree Regression**

In [11]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state = 42) #Estou utilizando os parâmetros achados no projeto 2 de regressão
dt.fit(x_train, y_train)

#Predição de treino
y_pred_train = dt.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = dt.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.010221817319147267
Erro do test set: 0.09641414682168067
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.6937102518956504
MSE do modelo: 0.09641414682168067
MAE do modelo: 0.23172853644321317


## **Random Forest Regression**

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth = 20, min_samples_leaf = 5, random_state = 42)
rf.fit(x_train, y_train)

#Predição de treino
y_pred_train = rf.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = rf.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.03469048703259924
Erro do test set: 0.06256548805840637
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.8012411227070491
MSE do modelo: 0.06256548805840637
MAE do modelo: 0.19288360142959873


## **Ridge Regression**

In [13]:
from sklearn.linear_model import Ridge
rdg = Ridge(alpha = 0.01)
rdg.fit(x_train, y_train)

#Predição de treino
y_pred_train = rdg.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = rdg.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))
print("Intercept:", rdg.intercept_)

Erro do train set: 0.045196339663549405
Erro do test set: 0.0774425320969118
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.753979530696918
MSE do modelo: 0.0774425320969118
MAE do modelo: 0.21673045235510968
Intercept: 4.942056385614376


## **LASSO Regression**

In [14]:
from sklearn.linear_model import Lasso
lss = Lasso(alpha = 0.001, max_iter = 100000)
lss.fit(x_train, y_train)

#Predição de treino
y_pred_train = lss.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = lss.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test,y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))
print("Intercept:", lss.intercept_)

Erro do train set: 0.06464380274935122
Erro do test set: 0.07883783355105665
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.7495469184194887
MSE do modelo: 0.07883783355105665
MAE do modelo: 0.21843307626687364
Intercept: -1.6689646506579225


## **Gradient Boosting Regression**

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(x_train, y_train)

#Predição de treino
y_pred_train = gb.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = gb.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test, y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.012801408906124178
Erro do test set: 0.0566038108710126
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.8201802583443067
MSE do modelo: 0.0566038108710126
MAE do modelo: 0.17978813727822018


# **Resultados dos modelos**
Como podemos ver, o Gradient Boosting Regressor foi o melhor modelo aplicado. Como podemos ver, o MSE no set de teste foi bem maior que no set de treino, isso indica que o modelo está com problema de overfitting. Vejamos como melhorá-lo.

In [16]:
gb = GradientBoostingRegressor(min_samples_split = 5, min_samples_leaf = 4) #Aumentamos arbitráriamente alguns valores para ver se melhora.
gb.fit(x_train, y_train)

#Predição de treino
y_pred_train = gb.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = gb.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test, y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.012512003491648386
Erro do test set: 0.05941919634391788
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.8112363042074965
MSE do modelo: 0.05941919634391788
MAE do modelo: 0.18355177402581185


In [27]:
#Vamos ver se melhoramos o modelo.
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

model = GradientBoostingRegressor()
parameters = {'learning_rate': sp_randFloat(),
              'subsample': sp_randFloat(),
              'n_estimators': sp_randInt(100, 1000),
              'max_depth': sp_randInt(4, 10),
              'min_samples_split': sp_randInt(2, 20),
              'min_samples_leaf': sp_randInt(1,20)}
rdm = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=5, n_iter=10, n_jobs=-1)
rdm.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002562EB3EF90>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002562F125580>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002562F126D20>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002562F127080>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002562EB060F0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002562EACEBD0>})

In [29]:
#Vamos ver os resultados
print("\n===========================================================================")
print("Results from Random Search\n")
print("Best estimator across ALL searched params:\n", rdm.best_estimator_)
print("Best score across ALL searched params:\n", rdm.best_score_)
print("Best parameters across ALL searched params:\n", rdm.best_params_)
print("===========================================================================")



Results from Random Search

Best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.0726426706614971, max_depth=8,
                          min_samples_leaf=5, min_samples_split=16,
                          n_estimators=982, subsample=0.908974100293609)
Best score across ALL searched params:
 0.6422905418499713
Best parameters across ALL searched params:
 {'learning_rate': 0.0726426706614971, 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 16, 'n_estimators': 982, 'subsample': 0.908974100293609}


In [30]:
#Vamos ver com Grid Search
model = GradientBoostingRegressor()

from sklearn.model_selection import GridSearchCV
grid = {'max_depth': [2, 3, 5, 10],
        'subsample': [0.9, 0.5, 0.2, 0.1],
        'n_estimators': [10, 20, 40, 100],
        'learning_rate': [0.01, 0.02, 0.03, 0.04],
        'min_samples_split': [2, 10, 15, 20],
        'min_samples_leaf': [1, 10, 15, 20]}
GBRegressor = GridSearchCV(estimator=model,param_grid=grid, cv=5, n_jobs=-1)
GBRegressor.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 10, 15, 20],
                         'min_samples_split': [2, 10, 15, 20],
                         'n_estimators': [10, 20, 40, 100],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [31]:
#Vamos ver os resultados
print("\n===========================================================================")
print("Results from Grid Search\n")
print("Best estimator across ALL searched params:\n", GBRegressor.best_estimator_)
print("Best score across ALL searched params:\n", GBRegressor.best_score_)
print("Best parameters across ALL searched params:\n", GBRegressor.best_params_)
print("===========================================================================")


Results from Grid Search

Best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.03, max_depth=5, min_samples_split=10,
                          subsample=0.5)
Best score across ALL searched params:
 0.7147127284408652
Best parameters across ALL searched params:
 {'learning_rate': 0.03, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100, 'subsample': 0.5}


In [32]:
#Vamos testar o Random Search
#Train-Test Split
y = data_materials['Exciton Binding Energy']
x = data_materials.drop(['Material', 'Space group','Exciton Binding Energy', 'Stoichiometry', 'Crystal Type'] , axis = 'columns')

#Criando os dados de teste e treino:
x_train, x_test, y_train, y_test = tts(x, y , test_size = 0.20,random_state = 42)

#Modelo
gb =  GradientBoostingRegressor(min_samples_split = 10, min_samples_leaf = 1,  
                          learning_rate=0.018858997890504936, max_depth=6,
                          n_estimators=417, subsample=0.4575658142824387)
gb.fit(x_train, y_train)

#Predição de treino
y_pred_train = gb.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = gb.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test, y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.012542150991250228
Erro do test set: 0.07090111847736984
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.7658518651697469
MSE do modelo: 0.07090111847736984
MAE do modelo: 0.19455229256217352


In [34]:
#Vamos testar o Grid Search
#Train-Test Split
y = data_materials['Exciton Binding Energy']
x = data_materials.drop(['Material', 'Space group','Exciton Binding Energy', 'Stoichiometry', 'Crystal Type'] , axis = 'columns')

#Criando os dados de teste e treino:
x_train, x_test, y_train, y_test = tts(x, y , test_size = 0.20,random_state = 42)

#Modelo
gb = GradientBoostingRegressor(learning_rate=0.03, max_depth=5, min_samples_split=10,
                          subsample=0.5)
gb.fit(x_train, y_train)

#Predição de treino
y_pred_train = gb.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = gb.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test, y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.0215637421118774
Erro do test set: 0.06434548340026836
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.787501590292504
MSE do modelo: 0.06434548340026836
MAE do modelo: 0.19060617581732006


In [35]:
#O do Grid Search se saiu melhor, mas ainda temos que melhorar. Vamos tentar com Bagging.
model_gb = GradientBoostingRegressor(learning_rate=0.04, subsample=0.9)

from sklearn.ensemble import BaggingRegressor
bag_gb = BaggingRegressor(estimator=model_gb, random_state=42)
bag_gb.fit(x_train, y_train)

#Treino
y_pred_train_bag=bag_gb.predict(x_train)
#Teste
y_pred_test_bag=bag_gb.predict(x_test)

#MSE
print("Train MSE:",mse(y_train, y_pred_train_bag))
print("Test MSE:",mse(y_test, y_pred_test_bag))
print("R2 Score:", r_2(y_test, y_pred_test_bag))

Train MSE: 0.03243682385796457
Test MSE: 0.0625462256970029
R2 Score: 0.7934435675750351


In [61]:
y_pred_test_df=pd.DataFrame(data=y_pred_test_bag)
y_pred_test_df.iloc[0]

0    1.132974
Name: 0, dtype: float64

In [62]:
y_test.iloc[0]

1.3067289723144695

Podemos ver que, sem tirar feature do dataset utilizando métodos para checar a importância delas, o melhor que conseguimos chegar foi esse modelo (a menor diferença entre o Train MSE e Test MSE e o maior R2). Vamos salvá-lo para poder testar com novos materiais.

In [63]:
#Salvando o modelo.
import pickle
final_model = bag_gb
final_model.fit(x_train, y_train)

#Salvando o modelo
filename = 'modelo_final_regressão.sav'
pickle.dump(final_model, open(filename, 'wb'))

## **Testando o Modelo com novos materiais**
Agora, criaremos uma lista com novos materiais para testarmos o modelo. Vamos fazer com elementos que geralmente constituem uma célula solar orgânica

In [64]:
df_atoms = pd.read_csv('./Schleder2019_AtomicTable.csv')
df_atoms

,Element,Z,Electronegativity,IonizationPotential,ElectronAffinity,HOMO,LUMO,r_s_orbital,r_p_orbital,r_d_orbital,r_atomic_nonbonded,r_valence_lastorbital,r_covalent,Valence,PeriodicColumn,PeriodicColumn_upto18,NumberUnfilledOrbitals,Polarizability
0,H,1,2.20,-12.6833,-1.5273,-6.4925,0.7250,0.3865,NaN,NaN,0.37,0.3865,0.31,1.0,1.0,1.0,1.0,4.507107
1,He,2,NaN,-26.7499,3.0204,-15.7610,1.5714,0.2964,1.0292,0.4176,0.32,0.2964,0.28,2.0,8.0,18.0,0.0,1.383746
2,Li,3,0.98,-5.3606,-0.5863,-2.8744,-0.9074,1.6578,1.8874,2.0869,1.34,1.6578,1.28,1.0,1.0,1.0,1.0,164.000000
3,Be,4,1.57,-9.5007,0.7972,-5.6097,-2.0104,1.0805,1.2128,1.9594,0.90,1.0805,0.96,2.0,2.0,2.0,0.0,37.710000
4,B,5,2.04,-8.1261,0.0312,-3.6067,2.4547,0.8025,0.8348,1.3619,0.82,0.8348,0.84,3.0,3.0,13.0,5.0,20.530000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Fl,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.590000
114,Mc,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,Lv,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,Ts,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_atoms.set_index('Element', inplace = True)
dicio = df_atoms.to_dict('index')

In [66]:
#Propriedades
prop = ['Z',
        'Electronegativity',
        'IonizationPotential',
        'ElectronAffinity',
        'HOMO',
        'LUMO',
        'r_s_orbital',
        'r_p_orbital',
        'r_d_orbital',
        'r_atomic_nonbonded',
        'r_valence_lastorbital',
        'r_covalent',
        'Valence',
        'PeriodicColumn',
        'PeriodicColumn_upto18',
        'NumberUnfilledOrbitals',
        'Polarizability']

In [67]:
data = ase.db.connect('./c2db-2021-06-24.db')
rows = data.select(is_magnetic=False) #Seleciona materias não metálicos e não magnéticos

#Listas de propriedades
lista = []
pesos = []
stch = []

#Features estatísticas
media_interm = {}

#Lista que guarda todas as informações
lista_completa = []

for row in rows:
    try:
        comp = Composition(row.formula).as_dict()
        elem = list(comp.items())
        media_interm['Material'] = row.formula
        media_interm['Space group'] = row.spacegroup
        #media_interm['Exciton Binding Energy'] = row.E_B
        media_interm['Stoichiometry'] = row.stoichiometry
        media_interm['Crystal Type'] = row.crystal_type
    
        for i in prop:
            #Lista de propriedades de cada átomo
            for m in range(0, len(elem)):
                lista.append(dicio[elem[m][0]][i])
                pesos.append(elem[m][1])
                if (len(elem)==2):
                    stch.append(row.stoichiometry)
        
            #Valor médio
            media_interm[f'media_{i}'] = np.mean(lista)

    
            #édia ponderada
            avg = np.average(lista,weights=pesos)
            media_interm[f'media_pon_{i}'] = avg
    
            #Valor máximo e mínimo
            max_prop = max(lista)
            min_prop = min(lista)
            media_interm[f'max_{i}'] = max_prop
            media_interm[f'min_{i}'] = min_prop
    
            #Desvio padrão em relação a média
            media_interm[f'desvio_{i}'] = np.std(lista)
    
            #Desvio padrão em relação a média ponderada
            sum_prop = 0
            for j in lista:
                sub2 = (j - avg)**2
                sum_prop = sum_prop + sub2
            media_interm[f'desvio_pon_{i}'] = np.sqrt(sum_prop/len(lista)) 
        
            lista.clear()
            pesos.clear()
        
        lista_completa.append(media_interm.copy())
    except:
        pass

print(set(stch))
    
print(len(lista_completa))
df_new = pd.DataFrame(lista_completa)
df_new

{'AB12', 'AB2', 'A2B3', 'AB3', 'AB5', 'A3B4', 'A2B5', 'AB4', 'AB'}
3227


,Material,Space group,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Be4,Pbcm,A,A-57-d,4.000000,4.000000,4,4,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,37.710000,37.710000,37.71,37.71,0.000000,0.000000
1,AlTe4,Cm,AB4,AB4-8-a,32.500000,44.200000,52,13,19.500000,22.740712,...,5.0,2.0,1.500000,1.749286,46.200000,40.680000,55.40,37.00,9.200000,10.728951
2,As4O6,P2_1,A2B3,A2B3-4-a,20.500000,18.000000,33,8,12.500000,12.747549,...,3.0,2.0,0.500000,0.509902,17.520000,15.064000,29.80,5.24,12.280000,12.523192
3,As4S6,Pc,A2B3,A2B3-7-a,24.500000,22.800000,33,16,8.500000,8.668333,...,3.0,2.0,0.500000,0.509902,24.585000,23.542000,29.80,19.37,5.215000,5.318277
4,B2N,P-3m1,AB2,AB2-164-bd,6.000000,5.666667,7,5,1.000000,1.054093,...,5.0,3.0,1.000000,1.054093,14.065000,16.220000,20.53,7.60,6.465000,6.814708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3222,Hf2Ti2Se8,P1,ABC4,ABC4-1-a,42.666667,38.333333,72,22,21.312490,21.748563,...,8.0,2.0,2.828427,3.464102,75.746667,50.993333,109.00,26.24,35.687837,43.432122
3223,Hf2Zr2Te8,P1,ABC4,ABC4-1-a,54.666667,53.333333,72,40,13.199327,13.266499,...,8.0,2.0,2.828427,3.464102,89.000000,63.000000,121.00,37.00,37.094474,45.299007
3224,Ti2Zr2S8,P1,ABC4,ABC4-1-a,26.000000,21.000000,40,16,10.198039,11.357817,...,8.0,2.0,2.828427,3.464102,77.456667,48.413333,121.00,19.37,42.745726,51.678935
3225,FeHfF6,P1,ABC6,ABC6-1-a,35.666667,19.000000,72,9,26.612445,31.400637,...,8.0,1.0,2.867442,3.544362,56.900000,23.650000,109.00,3.70,42.995581,54.352392


In [68]:
#Checando os tipos de estequiometria no nosso dataset original
stoichiometry = data_materials['Stoichiometry'].to_numpy()
def Repeat(x):
    _size = len(x)
    repeated = []
    for i in range(_size):
        k = i + 1
        for j in range(k, _size):
            if x[i] == x[j] and x[i] not in repeated:
                repeated.append(x[i])
    return repeated
 
#Vamos checar quais tipos temos
print ("As estequiometrias são:", Repeat(stoichiometry))

As estequiometrias são: ['A', 'AB', 'AB2', 'ABC']


In [69]:
#Vamos montar um data set só com elas usando o c2db

#Criando para cada estequiometria
df_A = df_new[(df_new == 'A').any(axis=1)]
df_AB = df_new[(df_new == 'AB').any(axis=1)]
df_AB2 = df_new[(df_new == 'AB2').any(axis=1)]
df_ABC = df_new[(df_new == 'ABC').any(axis=1)]

df_new_materials = pd.concat([df_A, df_AB, df_AB2, df_ABC], ignore_index=True)
df_new_materials

,Material,Space group,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Be4,Pbcm,A,A-57-d,4.000000,4.000000,4,4,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,37.710000,37.710000,37.71,37.71,0.000000,0.000000
1,As4,Pmna,A,A-53-h,33.000000,33.000000,33,33,0.000000,0.000000,...,3.0,3.0,0.000000,0.000000,29.800000,29.800000,29.80,29.80,0.000000,0.000000
2,Ge2,P-3m1,A,A-164-d,32.000000,32.000000,32,32,0.000000,0.000000,...,4.0,4.0,0.000000,0.000000,39.430000,39.430000,39.43,39.43,0.000000,0.000000
3,Si2,P-3m1,A,A-164-d,14.000000,14.000000,14,14,0.000000,0.000000,...,4.0,4.0,0.000000,0.000000,37.310000,37.310000,37.31,37.31,0.000000,0.000000
4,P2,P-3m1,A,A-164-d,15.000000,15.000000,15,15,0.000000,0.000000,...,3.0,3.0,0.000000,0.000000,24.930000,24.930000,24.93,24.93,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182,WSSe,P3m1,ABC,ABC-156-ac,41.333333,41.333333,74,16,24.239545,24.239545,...,6.0,2.0,1.885618,1.885618,40.203333,40.203333,75.00,19.37,24.764292,24.764292
2183,TaSTe,P3m1,ABC,ABC-156-abc,47.000000,47.000000,73,16,23.537205,23.537205,...,7.0,2.0,2.357023,2.357023,48.123333,48.123333,88.00,19.37,29.101153,29.101153
2184,ZrSTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,16,14.966630,14.966630,...,8.0,2.0,2.828427,2.828427,59.123333,59.123333,121.00,19.37,44.341445,44.341445
2185,TiSeTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,22,12.328828,12.328828,...,8.0,2.0,2.828427,2.828427,51.746667,51.746667,92.00,26.24,28.800377,28.800377


In [70]:
#Agora vamos retirar os que já foram usados.
new_materials = df_new_materials[~df_new_materials.index.isin(data_materials.index)]
new_materials.dropna(inplace = True)
new_materials

C:\Users\andre\AppData\Local\Temp\ipykernel_18868\535224484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_materials.dropna(inplace = True)


,Material,Space group,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
14,Te2,P-3m1,A,A-164-d,52.000000,52.000000,52,52,0.000000,0.000000,...,2.0,2.0,0.000000,0.000000,37.000000,37.000000,37.00,37.00,0.000000,0.000000
94,Ga2O2,Cmmm,AB,AB-65-hj,19.500000,19.500000,31,8,11.500000,11.500000,...,5.0,2.0,1.500000,1.500000,28.320000,28.320000,51.40,5.24,23.080000,23.080000
104,Hg2I2,P1,AB,AB-1-a,66.500000,66.500000,80,53,13.500000,13.500000,...,1.0,0.0,0.500000,0.500000,34.435000,34.435000,34.60,34.27,0.165000,0.165000
188,SnTe,P3m1,AB,AB-156-bc,51.000000,51.000000,52,50,1.000000,1.000000,...,4.0,2.0,1.000000,1.000000,46.670000,46.670000,56.34,37.00,9.670000,9.670000
333,Re2Se2,P-6m2,AB,AB-187-hi,54.500000,54.500000,75,34,20.500000,20.500000,...,5.0,2.0,1.500000,1.500000,45.620000,45.620000,65.00,26.24,19.380000,19.380000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182,WSSe,P3m1,ABC,ABC-156-ac,41.333333,41.333333,74,16,24.239545,24.239545,...,6.0,2.0,1.885618,1.885618,40.203333,40.203333,75.00,19.37,24.764292,24.764292
2183,TaSTe,P3m1,ABC,ABC-156-abc,47.000000,47.000000,73,16,23.537205,23.537205,...,7.0,2.0,2.357023,2.357023,48.123333,48.123333,88.00,19.37,29.101153,29.101153
2184,ZrSTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,16,14.966630,14.966630,...,8.0,2.0,2.828427,2.828427,59.123333,59.123333,121.00,19.37,44.341445,44.341445
2185,TiSeTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,22,12.328828,12.328828,...,8.0,2.0,2.828427,2.828427,51.746667,51.746667,92.00,26.24,28.800377,28.800377


In [71]:
#Vamos usar o modelo
#Agora, vamos usar o modelo:
features = new_materials.drop(['Material','Space group','Stoichiometry','Crystal Type'],axis='columns')

#Carregamos o modelo
loaded_model = pickle.load(open(filename, 'rb'))

#Usando ele
final_pred = loaded_model.predict(features)

In [72]:
new_materials_Eb = pd.DataFrame(final_pred, columns = ['Exciton Binding Energy'])
new_materials_Eb.info()
new_materials_Eb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Exciton Binding Energy  1830 non-null   float64
dtypes: float64(1)
memory usage: 14.4 KB


,Exciton Binding Energy
0,0.472222
1,1.227857
2,0.889303
3,0.422880
4,0.555151
...,...
1825,0.577711
1826,0.508052
1827,0.546691
1828,0.520908


In [73]:
new_materials_df = new_materials.reset_index()
new_materials_df.drop('index', axis = 'columns', inplace = True)
new_materials_df

,Material,Space group,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Te2,P-3m1,A,A-164-d,52.000000,52.000000,52,52,0.000000,0.000000,...,2.0,2.0,0.000000,0.000000,37.000000,37.000000,37.00,37.00,0.000000,0.000000
1,Ga2O2,Cmmm,AB,AB-65-hj,19.500000,19.500000,31,8,11.500000,11.500000,...,5.0,2.0,1.500000,1.500000,28.320000,28.320000,51.40,5.24,23.080000,23.080000
2,Hg2I2,P1,AB,AB-1-a,66.500000,66.500000,80,53,13.500000,13.500000,...,1.0,0.0,0.500000,0.500000,34.435000,34.435000,34.60,34.27,0.165000,0.165000
3,SnTe,P3m1,AB,AB-156-bc,51.000000,51.000000,52,50,1.000000,1.000000,...,4.0,2.0,1.000000,1.000000,46.670000,46.670000,56.34,37.00,9.670000,9.670000
4,Re2Se2,P-6m2,AB,AB-187-hi,54.500000,54.500000,75,34,20.500000,20.500000,...,5.0,2.0,1.500000,1.500000,45.620000,45.620000,65.00,26.24,19.380000,19.380000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,WSSe,P3m1,ABC,ABC-156-ac,41.333333,41.333333,74,16,24.239545,24.239545,...,6.0,2.0,1.885618,1.885618,40.203333,40.203333,75.00,19.37,24.764292,24.764292
1826,TaSTe,P3m1,ABC,ABC-156-abc,47.000000,47.000000,73,16,23.537205,23.537205,...,7.0,2.0,2.357023,2.357023,48.123333,48.123333,88.00,19.37,29.101153,29.101153
1827,ZrSTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,16,14.966630,14.966630,...,8.0,2.0,2.828427,2.828427,59.123333,59.123333,121.00,19.37,44.341445,44.341445
1828,TiSeTe,P3m1,ABC,ABC-156-ac,36.000000,36.000000,52,22,12.328828,12.328828,...,8.0,2.0,2.828427,2.828427,51.746667,51.746667,92.00,26.24,28.800377,28.800377


In [74]:
final_predicted_Eb = pd.concat([new_materials_df['Material'], new_materials_Eb], axis=1)
final_predicted_Eb

,Material,Exciton Binding Energy
0,Te2,0.472222
1,Ga2O2,1.227857
2,Hg2I2,0.889303
3,SnTe,0.422880
4,Re2Se2,0.555151
...,...,...
1825,WSSe,0.577711
1826,TaSTe,0.508052
1827,ZrSTe,0.546691
1828,TiSeTe,0.520908


In [75]:
final_predicted_Eb.loc[final_predicted_Eb['Material'].str.contains('GaAs')]

,Material,Exciton Binding Energy
245,GaAs,0.490673
